In [ ]:
import pymysql
from web3 import Web3

In [ ]:
# Alchemy API endpoint
alchemy_url = "https://eth-mainnet.alchemyapi.io/v2/your_alchemy_api_key"  # Update with your Alchemy API endpoint

# Database connection settings
db_host = "localhost"  # Update with your database host
db_user = "root"  # Update with your database username
db_password = "E4s7b0urn3ru9by#!"  # Update with your database password
db_name = "ethereum block 11,473,329"  # Update with your database name

In [ ]:
# Ethereum block number to fetch
block_number = 11473329  # Ethereum blockchain, block number 11,473,329

# Connect to the Alchemy API
w3 = Web3(Web3.HTTPProvider(alchemy_url))

# Connect to the database
db = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name)
cursor = db.cursor()

try:
    # Fetch the block data from Alchemy
    block = w3.eth.get_block(block_number)

    # Extract all relevant data from the block
    block_data = {}
    for key in block:
        value = getattr(block, key)
        if isinstance(value, bytes):
            value = value.hex()
        elif key in ['transactions', 'uncles']:
            value = ','.join(value)
        block_data[key] = value

    # Prepare the column names and placeholders for the query
    columns = ', '.join(block_data.keys())
    placeholders = ', '.join(['%s'] * len(block_data))

    # Prepare the values for the query
    values = tuple(block_data.values())

    # Insert the block data into the database
    query = f"INSERT INTO blocks ({columns}) VALUES ({placeholders})"
    cursor.execute(query, values)


    # Commit the transaction
    db.commit()

# Close the database connection
finally:
    cursor.close()
    db.close()